## Import standard libraries

In [1]:
import os
import sys
# append coeqwal packages to path
sys.path.append('./coeqwalpackage')
import datetime as dt
import pandas as pd
import numpy as np
import cqwlutils as cu

## Import custom modules

In [2]:
# Import custom modules - NEED WINDOWS OS (NOTE: I had to run this twice, must check why this happens!)
from coeqwalpackage.DataExtraction import *
from coeqwalpackage.metrics import *


## Define contol file name

In [3]:
CtrlFile = 'CalSim3GroundWaterDataExtractionInitFile_v1.xlsx'
CtrlTab = 'Init'

## Read from control file

In [4]:
ScenarioListFile, ScenarioListTab, ScenarioListPath, GW1DssNamesOutPath, GW2DssNamesOutPath, ScenarioIndicesOutPath, DssDirsOutPath, VarListPath, VarListFile, VarListTab, VarOutPath, DataOutPath, ConvertDataOutPath, ExtractionSubPath, DemandDeliverySubPath, ModelSubPath, GroupDataDirPath, ScenarioDir, GW1DssMin, GW1DssMax, GW2DssMin, GW2DssMax, NameMin, NameMax, DirMin, DirMax, IndexMin, IndexMax, StartMin, StartMax, EndMin, EndMax, VarMin, VarMax, DemandFilePath, DemandFileName, DemandFileTab, DemMin, DemMax, InflowOutSubPath, InflowFilePath, InflowFileName, InflowFileTab, InflowMin, InflowMax = cu.read_init_file(CtrlFile, CtrlTab)

In [5]:
print([ScenarioListFile, ScenarioListTab, ScenarioListPath, GW1DssNamesOutPath, GW2DssNamesOutPath, ScenarioIndicesOutPath, DssDirsOutPath, VarListPath, VarListFile, VarListTab, VarOutPath, DataOutPath, ConvertDataOutPath, ExtractionSubPath, DemandDeliverySubPath, ModelSubPath, GroupDataDirPath, ScenarioDir, GW1DssMin, GW1DssMax, GW2DssMin, GW2DssMax, NameMin, NameMax, DirMin, DirMax, IndexMin, IndexMax, StartMin, StartMax, EndMin, EndMax, VarMin, VarMax, DemandFilePath, DemandFileName, DemandFileTab, DemMin, DemMax, InflowOutSubPath, InflowFilePath, InflowFileName, InflowFileTab, InflowMin, InflowMax])


['coeqwal_GW_scenario_listing_v1.xlsx', 'scenario_list', '../../CalSim3_Model_Runs/Scenarios\\coeqwal_GW_scenario_listing_v1.xlsx', '../../CalSim3_Model_Runs/Scenarios\\Group_Data_Extraction\\DVDssNamesFrom_coeqwal_GW_scenario_listing_v1.csv', '../../CalSim3_Model_Runs/Scenarios\\Group_Data_Extraction\\SVDssNamesFrom_coeqwal_GW_scenario_listing_v1.csv', '../../CalSim3_Model_Runs/Scenarios\\Group_Data_Extraction\\IndicesFrom_coeqwal_GW_scenario_listing_v1.csv', '../../CalSim3_Model_Runs/Scenarios\\Group_Data_Extraction\\DirNamesFrom_coeqwal_GW_scenario_listing_v1.csv', '../../CalSim3_Model_Runs/Scenarios\\ground_water_variables_v1.xlsx', 'ground_water_variables_v1.xlsx', 'GroundWaterVars_CS3', '../../CalSim3_Model_Runs/Scenarios\\Group_Data_Extraction\\VarsFrom_ground_water_variables_v1.csv', '../../CalSim3_Model_Runs/Scenarios\\Group_Data_Extraction\\DataFrom_ground_water_variables_v1.csv', '../../CalSim3_Model_Runs/Scenarios\\Group_Data_Extraction\\ConvertDataFrom_ground_water_variabl

## Check for output directory and create if necessary (not necessary)

In [6]:
# check if output directory exists
if not os.path.exists(GroupDataDirPath):
    # print warning
    print("Warning: directory " + GroupDataDirPath + " does not exists and will be created")
    
    # Create the directory
    os.makedirs(GroupDataDirPath)


## Define Nan Values

In [7]:
# NaN values as defined by CalSim3
Nan1 = -901
Nan2 = -902

## Read indeces, dss names, directory names, start and end dates, time range (not necessary)

In [8]:
gw1dsshdr, gw1dssname = cu.read_from_excel(ScenarioListPath, ScenarioListTab, GW1DssMin, GW1DssMax, hdr=True)
gw1dss_names = []
for i in range(len(gw1dssname)):
    gw1dss_names.append(gw1dssname[i][0])
gw1dss_names

['s0001_DCR2023_9.3.1_danube_hist\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0002_DCR2023_9.3.1_danube_adj\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0003_DCR2023_9.3.1_danube_cc50\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0004_DCR2023_9.3.1_danube_cc75\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0005_DCR2023_9.3.1_danube_cc95\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0006_draftDCP_NDDOff_2020\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0007_draftDCP_NDD_SWP3000_2020\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0008_draftDCP_NDD_SWP4500_2020\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0009_draftDCP_NDD_SWP6000_2020\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0010_draftDCP_NDD_SWP6000_Beth_2020\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0011_adjBL_wTUCP\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0012_adj_SGMApump\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0013_DCR-CC75_wTU

In [9]:
gw2dsshdr, gw2dssname = cu.read_from_excel(ScenarioListPath, ScenarioListTab, GW2DssMin, GW2DssMax, hdr=True)
gw2dss_names = []
for i in range(len(gw2dssname)):
    gw2dss_names.append(gw2dssname[i][0])
gw2dss_names

['s0001_DCR2023_9.3.1_danube_hist\\Model_Files\\DSS\\output\\CVGroundwaterBudget.dss',
 's0002_DCR2023_9.3.1_danube_adj\\Model_Files\\DSS\\output\\CVGroundwaterBudget.dss',
 's0003_DCR2023_9.3.1_danube_cc50\\Model_Files\\DSS\\output\\CVGroundwaterBudget.dss',
 's0004_DCR2023_9.3.1_danube_cc75\\Model_Files\\DSS\\output\\CVGroundwaterBudget.dss',
 's0005_DCR2023_9.3.1_danube_cc95\\Model_Files\\DSS\\input\\output\\CVGroundwaterBudget.dss',
 's0006_draftDCP_NDDOff_2020\\Model_Files\\DSS\\output\\CVGroundwaterBudget.dss',
 's0007_draftDCP_NDD_SWP3000_2020\\Model_Files\\DSS\\output\\CVGroundwaterBudget.dss',
 's0008_draftDCP_NDD_SWP4500_2020\\Model_Files\\DSS\\output\\CVGroundwaterBudget.dss',
 's0009_draftDCP_NDD_SWP6000_2020\\Model_Files\\DSS\\output\\CVGroundwaterBudget.dss',
 's0010_draftDCP_NDD_SWP6000_Beth_2020\\Model_Files\\DSS\\output\\CVGroundwaterBudget.dss',
 's0011_adjBL_wTUCP\\Model_Files\\DSS\\output\\CVGroundwaterBudget.dss',
 's0012_adj_SGMApump\\Model_Files\\DSS\\output\\CVG

In [10]:
indexhdr, index_name = cu.read_from_excel(ScenarioListPath, ScenarioListTab, IndexMin, IndexMax, hdr=True)
index_names = []
for i in range(len(index_name)):
    index_names.append(index_name[i][0])
index_names

['s0001',
 's0002',
 's0003',
 's0004',
 's0005',
 's0006',
 's0007',
 's0008',
 's0009',
 's0010',
 's0011',
 's0012',
 's0013',
 's0014',
 's0015',
 's0016',
 's0018']

In [11]:
studyhdr, study_name = cu.read_from_excel(ScenarioListPath, ScenarioListTab, NameMin, NameMax, hdr=True)
study_names = []
for i in range(len(study_name)):
    study_names.append(study_name[i][0])
study_names

['9.3.1_danube_hist',
 '9.3.1_danube_adj',
 '9.3.1_danube_cc50',
 '9.3.1_danube_cc75',
 '9.3.1_danube_cc95',
 's0006_draftDCP_NDDOff_2020',
 's0007_draftDCP_NDD_SWP3000_2020',
 's0008_draftDCP_NDD_SWP4500_2020',
 's0009_draftDCP_NDD_SWP6000_2020',
 's0010_draftDCP_NDD_SWP6000_Beth_2020',
 's0011_adjBL_wTUCP',
 's0012_adj_SGMApump',
 's0013_DCR-CC75_wTUCP',
 's0014_DCR-CC75_SGMApump',
 's0015_SGMApump_wTUCP',
 's0016_DCR-CC75_SGMApump_wTUCP',
 's0018_adj_EflowsV1']

In [12]:
dirhdr, dir_name = cu.read_from_excel(ScenarioListPath, ScenarioListTab, DirMin, DirMax, hdr=True)
dir_names = []
for i in range(len(dir_name)):
    dir_names.append(dir_name[i][0])
dir_names

['s0001_DCR2023_9.3.1_danube_hist',
 's0002_DCR2023_9.3.1_danube_adj',
 's0003_DCR2023_9.3.1_danube_cc50',
 's0004_DCR2023_9.3.1_danube_cc75',
 's0005_DCR2023_9.3.1_danube_cc95',
 's0006_draftDCP_NDDOff_2020',
 's0007_draftDCP_NDD_SWP3000_2020',
 's0008_draftDCP_NDD_SWP4500_2020',
 's0009_draftDCP_NDD_SWP6000_2020',
 's0010_draftDCP_NDD_SWP6000_Beth_2020',
 's0011_adjBL_wTUCP',
 's0012_adj_SGMApump',
 's0013_DCR-CC75_wTUCP',
 's0014_DCR-CC75_SGMApump',
 's0015_SGMApump_wTUCP',
 's0016_DCR-CC75_SGMApump_wTUCP',
 's0018_adj_EflowsV1']

In [13]:
starthdr, start_date = cu.read_from_excel(ScenarioListPath, ScenarioListTab, StartMin, StartMax, hdr=True)
start_dates = []
for i in range(len(start_date)):
    start_dates.append(start_date[i][0])
datetime_start_dates = pd.to_datetime(start_dates)
# turns out that dss reading library wands a dt datetime, not pd datetime
dt_datetime_start_dates = [dt.to_pydatetime() for dt in datetime_start_dates]


In [14]:
endhdr, end_date = cu.read_from_excel(ScenarioListPath, ScenarioListTab, EndMin, EndMax, hdr=True)
end_dates = []
for i in range(len(end_date)):
    end_dates.append(end_date[i][0])
# turns out that dss reading library wands a dt datetime, not pd datetime
datetime_end_dates = pd.to_datetime(end_dates)
dt_datetime_end_dates = [dt.to_pydatetime() for dt in datetime_end_dates]


In [15]:
min_datetime = min(dt_datetime_start_dates)
print('Min time: ')
print(min_datetime)
max_datetime = max(dt_datetime_end_dates)
print('Max time: ')
print(max_datetime)


Min time: 
1921-10-31 00:00:00
Max time: 
2021-09-30 00:00:00


## Read variables list (not necessary)

In [16]:
# get vars
hdr, vars = cu.read_from_excel(VarListPath, VarListTab,VarMin,VarMax,hdr=True)
gw1var_df = pd.DataFrame(data=vars, columns=hdr)
gw1var_df

,Part B:,Part C:
0,LT:E999,PUMPING
1,SR1:L1,GW_STORAGE
2,SR2:L1,GW_STORAGE
3,SR3:L1,GW_STORAGE
4,SR4:L1,GW_STORAGE
...,...,...
194,SR62:L3,GW_STORAGE
195,SR63:L3,GW_STORAGE
196,SR64:L3,GW_STORAGE
197,SR65:L3,GW_STORAGE


## Read the compund data from CSV to df

In [17]:
# read the dataframe from CSV
print('Reading ' + DataOutPath)
gw1_df, gw1dss_names = read_in_df(DataOutPath,GW1DssNamesOutPath)

Reading ../../CalSim3_Model_Runs/Scenarios\Group_Data_Extraction\DataFrom_ground_water_variables_v1.csv


In [18]:
print("gw1dss_names:")
gw1dss_names

gw1dss_names:


['s0001_DCR2023_9.3.1_danube_hist\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0002_DCR2023_9.3.1_danube_adj\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0003_DCR2023_9.3.1_danube_cc50\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0004_DCR2023_9.3.1_danube_cc75\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0005_DCR2023_9.3.1_danube_cc95\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0006_draftDCP_NDDOff_2020\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0007_draftDCP_NDD_SWP3000_2020\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0008_draftDCP_NDD_SWP4500_2020\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0009_draftDCP_NDD_SWP6000_2020\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0010_draftDCP_NDD_SWP6000_Beth_2020\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0011_adjBL_wTUCP\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0012_adj_SGMApump\\Model_Files\\DSS\\output\\CVGroundwaterOut.dss',
 's0013_DCR-CC75_wTU

In [19]:
print("gw1_df:")
gw1_df

gw1_df:


IWFM                               \
                     LT:E999_s0001                SR10:L1_s0001   
                           PUMPING                   GW_STORAGE   
                              1MON                         1MON   
           PUMPING_AT_ALL_ELEMENTS GW_STORAGE_AT_CALSIM_REGIONS   
                           PER-CUM                      PER-CUM   
                               TAF                          TAF   
1921-10-31               -0.489879                    4565.6143   
1921-11-30               -0.175123                    4567.0170   
1921-12-31               -0.121138                    4577.3060   
1922-01-31               -0.107654                    4575.9834   
1922-02-28               -0.255931                    4592.6543   
..                             ...                          ...   
2021-05-31               -2.909803                    4515.9260   
2021-06-30               -2.372584                    4507.4830   
2021-07-31               -1.912345                    4501.7593   
2021-08-31               -2.008060                    4492.0356   
2021-09-30               -1.959299                    4488.5010   

                                                                      \
                          SR10:L2_s0001                SR10:L3_s0001   
                             GW_STORAGE                   GW_STORAGE   
                                   1MON                         1MON   
           GW_STORAGE_AT_CALSIM_REGIONS GW_STORAGE_AT_CALSIM_REGIONS   
                                PER-CUM                      PER-CUM   
                                    TAF                          TAF   
1921-10-31                    3738.8423                     9413.029   
1921-11-30                    3739.4534                     9413.407   
1921-12-31                    3740.1123                     9414.034   
1922-01-31                    3740.3958                     9414.589   
1922-02-28                    3741.3260                     9415.569   
..                                  ...                          ...   
2021-05-31                    3734.8535                     9407.857   
2021-06-30                    3733.3555                     9405.691   
2021-07-31                    3731.9446                     9403.306   
2021-08-31                    3731.9985                     9401.803   
2021-09-30                    3733.1638                     9401.524   

                                                                      \
                          SR11:L1_s0001                SR11:L2_s0001   
                             GW_STORAGE                   GW_STORAGE   
                                   1MON                         1MON   
           GW_STORAGE_AT_CALSIM_REGIONS GW_STORAGE_AT_CALSIM_REGIONS   
                                PER-CUM                      PER-CUM   
                                    TAF                          TAF   
1921-10-31                    4903.5234                    3620.7840   
1921-11-30                    4903.3374                    3622.1665   
1921-12-31                    4908.7720                    3623.2290   
1922-01-31                    4913.5996                    3624.0764   
1922-02-28                    4929.2583                    3625.1330   
..                                  ...                          ...   
2021-05-31                    4640.1797                    3610.0195   
2021-06-30                    4618.6553                    3606.8208   
2021-07-31                    4593.8164                    3603.9067   
2021-08-31                    4574.6504                    3603.1633   
2021-09-30                    4564.2050                    3604.3066   

                                                                      \
                          SR11:L3_s0001                SR12:L1_s0001   
                             GW_STORAGE                   GW_STORAGE   
             

## Add water year column to df

In [21]:
df = add_water_year_column(gw1_df)

C:\Dino\COEQWAL\COEQWAL_GIT\coeqwal\notebooks\coeqwalpackage\metrics.py:175: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  return df_copy.drop(["Date", "Year", "Month"], axis=1)


In [22]:
print("gw1_df with water year column:")
gw1_df

gw1_df with water year column:


IWFM                               \
                     LT:E999_s0001                SR10:L1_s0001   
                           PUMPING                   GW_STORAGE   
                              1MON                         1MON   
           PUMPING_AT_ALL_ELEMENTS GW_STORAGE_AT_CALSIM_REGIONS   
                           PER-CUM                      PER-CUM   
                               TAF                          TAF   
1921-10-31               -0.489879                    4565.6143   
1921-11-30               -0.175123                    4567.0170   
1921-12-31               -0.121138                    4577.3060   
1922-01-31               -0.107654                    4575.9834   
1922-02-28               -0.255931                    4592.6543   
..                             ...                          ...   
2021-05-31               -2.909803                    4515.9260   
2021-06-30               -2.372584                    4507.4830   
2021-07-31               -1.912345                    4501.7593   
2021-08-31               -2.008060                    4492.0356   
2021-09-30               -1.959299                    4488.5010   

                                                                      \
                          SR10:L2_s0001                SR10:L3_s0001   
                             GW_STORAGE                   GW_STORAGE   
                                   1MON                         1MON   
           GW_STORAGE_AT_CALSIM_REGIONS GW_STORAGE_AT_CALSIM_REGIONS   
                                PER-CUM                      PER-CUM   
                                    TAF                          TAF   
1921-10-31                    3738.8423                     9413.029   
1921-11-30                    3739.4534                     9413.407   
1921-12-31                    3740.1123                     9414.034   
1922-01-31                    3740.3958                     9414.589   
1922-02-28                    3741.3260                     9415.569   
..                                  ...                          ...   
2021-05-31                    3734.8535                     9407.857   
2021-06-30                    3733.3555                     9405.691   
2021-07-31                    3731.9446                     9403.306   
2021-08-31                    3731.9985                     9401.803   
2021-09-30                    3733.1638                     9401.524   

                                                                      \
                          SR11:L1_s0001                SR11:L2_s0001   
                             GW_STORAGE                   GW_STORAGE   
                                   1MON                         1MON   
           GW_STORAGE_AT_CALSIM_REGIONS GW_STORAGE_AT_CALSIM_REGIONS   
                                PER-CUM                      PER-CUM   
                                    TAF                          TAF   
1921-10-31                    4903.5234                    3620.7840   
1921-11-30                    4903.3374                    3622.1665   
1921-12-31                    4908.7720                    3623.2290   
1922-01-31                    4913.5996                    3624.0764   
1922-02-28                    4929.2583                    3625.1330   
..                                  ...                          ...   
2021-05-31                    4640.1797                    3610.0195   
2021-06-30                    4618.6553                    3606.8208   
2021-07-31                    4593.8164                    3603.9067   
2021-08-31                    4574.6504                    3603.1633   
2021-09-30                    4564.2050                    3604.3066   

                                                                      \
                          SR11:L3_s0001                SR12:L1_s0001   
                             GW_STORAGE                   GW_STORAGE   
             

## End of initialization

In [23]:
print('Done Initializing!')

Done Initializing!


## Compute trends in all variables SRxx:TOT, use start and end water year (default WY = 1960 to 2021), compute linear fit, get slope of line, write to file (note that some scenarios only go to 2015)